In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [2]:
# Load the dataset
data = pd.read_csv("final_dataset.csv")

In [3]:
# Define the image data generator with augmentation (optional)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
# Split the data into training and testing sets with an 80:20 ratio
train_data = data.sample(frac=0.8, random_state=42)
test_data = data.drop(train_data.index)

In [5]:
# Define the input image size and batch size
img_size = (224, 224)
batch_size = 32

In [6]:
# Create the generators for training and testing data
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory=None,
    x_col="local_filename",
    y_col="label",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory=None,
    x_col="local_filename",
    y_col="label",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
)

Found 9772 validated image filenames belonging to 114 classes.
Found 2443 validated image filenames belonging to 114 classes.


In [7]:
# Define the InceptionNet model and load pre-trained weights
base_model = InceptionV3(weights='imagenet', include_top=False)

In [8]:
# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

In [9]:
# Add a fully-connected layer with 1024 neurons
x = Dense(1024, activation='relu')(x)

In [10]:
# Add a final output layer with one neuron per class
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

In [11]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

In [12]:
# Freeze all layers in the base model for transfer learning
for layer in base_model.layers:
    layer.trainable = False

In [13]:
# Compile the model with categorical cross-entropy loss and Adam optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
# Train the model on the training data
history = model.fit(train_generator, epochs=20, validation_data=test_generator)

Epoch 1/20
306/306 [==============================] - 589s 2s/step - loss: 4.2682 - accuracy: 0.0820 - val_loss: 3.9626 - val_accuracy: 0.1036
Epoch 2/20
306/306 [==============================] - 584s 2s/step - loss: 3.7492 - accuracy: 0.1406 - val_loss: 3.7632 - val_accuracy: 0.1388
Epoch 3/20
306/306 [==============================] - 572s 2s/step - loss: 3.5083 - accuracy: 0.1788 - val_loss: 3.6645 - val_accuracy: 0.1527
Epoch 4/20
306/306 [==============================] - 571s 2s/step - loss: 3.3157 - accuracy: 0.2078 - val_loss: 3.6258 - val_accuracy: 0.1695
Epoch 5/20
306/306 [==============================] - 642s 2s/step - loss: 3.1542 - accuracy: 0.2350 - val_loss: 3.5696 - val_accuracy: 0.1854
Epoch 6/20
306/306 [==============================] - 695s 2s/step - loss: 3.0311 - accuracy: 0.2618 - val_loss: 3.5431 - val_accuracy: 0.1944
Epoch 7/20
306/306 [==============================] - 763s 2s/step - loss: 2.9204 - accuracy: 0.2804 - val_loss: 3.5793 - val_accuracy: 0.1948

In [15]:
# Evaluate the model on the testing data
test_loss, test_acc = model.evaluate(test_generator)
print("Test accuracy:", test_acc)

77/77 [==============================] - 111s 1s/step - loss: 3.8525 - accuracy: 0.2030
Test accuracy: 0.203029066324234


In [16]:
print('Test accuracy: {:.2f}%'.format(test_acc * 100))

Test accuracy: 20.30%


In [18]:
import numpy as np
# Load test set
test_df = pd.read_csv('final_dataset.csv')

# Create data generator for test set
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,
    x_col="local_filename",
    y_col="label",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Predict labels for test set
y_pred = model.predict(test_generator)
predicted_labels = np.argmax(y_pred, axis=1)
actual_labels = test_generator.classes

# Map label indices to label names
label_map = {v: k for k, v in test_generator.class_indices.items()}
predicted_labels = [label_map[label] for label in predicted_labels]
actual_labels = [label_map[label] for label in actual_labels]

# Get fitzpatrick_scale values for test set
fitzpatrick_scale = test_df['fitzpatrick_scale']

# Combine the predicted labels, actual labels, and fitzpatrick_scale values into a DataFrame
results_df = pd.DataFrame({'predicted_label': predicted_labels, 'actual_label': actual_labels, 'fitzpatrick_scale': fitzpatrick_scale})

# Save the DataFrame to a CSV file
results_df.to_csv('results_InceptionNet.csv', index=False)

# Print individual accuracies for each fitzpatrick scale value
for scale in range(1, 7):
    scale_df = results_df[results_df['fitzpatrick_scale'] == scale]
    total = len(scale_df)
    correct = sum(scale_df['predicted_label'] == scale_df['actual_label'])
    accuracy = correct / total
    print(f"Fitzpatrick Scale {scale}: {accuracy:.2%} ({correct}/{total})")


Found 12215 validated image filenames belonging to 114 classes.
382/382 [==============================] - 702s 2s/step
Fitzpatrick Scale 1: 1.33% (37/2777)
Fitzpatrick Scale 2: 1.62% (67/4136)
Fitzpatrick Scale 3: 1.44% (33/2295)
Fitzpatrick Scale 4: 1.12% (19/1689)
Fitzpatrick Scale 5: 1.01% (9/891)
Fitzpatrick Scale 6: 1.87% (8/427)


In [19]:
#UPDATED CODE
import pandas as pd

# read csv file into a pandas dataframe
df = pd.read_csv("results_InceptionNet.csv")

# compute total count of fitzpatrick_scale
total_count = df.shape[0]

print(total_count)

# compute count of correctly predicted values
correctly_predicted = df[df["actual_label"] == df["predicted_label"]].shape[0]
print(correctly_predicted)

answer = (correctly_predicted/total_count)*100

print(answer)

12215
173
1.4162914449447401


In [22]:
accuracy = 20.3
d_accuracy = 1.41
new = accuracy/d_accuracy

f=[1.33,1.62,1.44,1.12,1.01,1.87]
l = [None] * 6

for i in range(0,6):
    l[i]=f[i]*new
print(l)

[19.148226950354612, 23.323404255319154, 20.731914893617024, 16.124822695035466, 14.54113475177305, 26.922695035460997]
